# Optimize Arbitrage

## Setup
**Do NOT click the button at the top to run all cells!**

This is an interactive document consisting of text parts (like this one) and code parts (with a greyish background). Click in the next line and press **Shift + Enter** to execute the code.

In [1]:
import importlib
from IPython.display import clear_output

if importlib.util.find_spec("pulp") is None :
    %pip install pulp
    clear_output(wait=True) 

import pulp
from pulp import LpVariable
from pulp import LpProblem 
from pulp import *

In [3]:
#game data
rarities = [2,1.6,1.4,1.2]
all_products = { 120034: {"guid": 120034, "name": "Corn", "agio": 1, "weight": 265.47999799999997}, 112694: { "guid": 112694, "name": "Caribou Meat", "agio": 1, "weight": 645.70604 }, 112697: { "guid": 112697, "name": "Goose Feathers", "agio": 1.4, "weight": 184.308456 }, 114357: { "guid": 114357, "name": "Sanga Cow", "agio": 1, "weight": 136.8545464 }, 114369: { "guid": 114369, "name": "Spices", "agio": 1.4, "weight": 203.0659098 }, 114370: { "guid": 114370, "name": "Beeswax", "agio": 1, "weight": 162.3818196 }, 114390: { "guid": 114390, "name": "Hibiscus Tea", "agio": 1.2, "weight": 279.086366 }, 114391: { "guid": 114391, "name": "Linen", "agio": 1.2, "weight": 178.63181960000003 }, 114404: { "guid": 114404, "name": "Tapestries", "agio": 1.2, "weight": 914.1698670000001 }, 114410: { "guid": 114410, "name": "Seafood Stew", "agio": 1.4, "weight": 1353.99171726 }, 114414: { "guid": 114414, "name": "Clay Pipes", "agio": 1.4, "weight": 1161.0965455 }, 114428: { "guid": 114428, "name": "Leather Boots", "agio": 1.4, "weight": 554.2391464 }, 114430: { "guid": 114430, "name": "Tailored Suits", "agio": 1.2, "weight": 983.7046155999999 }, 114431: { "guid": 114431, "name": "Telephones", "agio": 1.6, "weight": 2926.4020599999994 }, 120008: { "guid": 120008, "name": "Wood", "agio": 1, "weight": 69.33594 }, 120016: { "guid": 120016, "name": "Champagne", "agio": 1.6, "weight": 1013.6174225 }, 120030: { "guid": 120030, "name": "Glasses", "agio": 1.4, "weight": 996.6566975000001 }, 120032: { "guid": 120032, "name": "Coffee", "agio": 1.4, "weight": 920.9361979999999 }, 120033: { "guid": 120033, "name": "Fried Plantains", "agio": 1, "weight": 250.70598909999995 }, 120035: { "guid": 120035, "name": "Tortillas", "agio": 1, "weight": 853.494558 }, 120037: { "guid": 120037, "name": "Bowler Hats", "agio": 1.4, "weight": 449.4541519999999 }, 120041: { "guid": 120041, "name": "Plantains", "agio": 1, "weight": 113.57999799999999 }, 120044: { "guid": 120044, "name": "Felt", "agio": 1.2, "weight": 78.05999599999998 }, 1010192: { "guid": 1010192, "name": "Grain", "agio": 1, "weight": 177.34376 }, 1010193: { "guid": 1010193, "name": "Beef", "agio": 1, "weight": 157.34376 }, 1010194: { "guid": 1010194, "name": "Hops", "agio": 1.2, "weight": 183.34376 }, 1010195: { "guid": 1010195, "name": "Potatoes", "agio": 1, "weight": 61.84376 }, 1010196: { "guid": 1010196, "name": "Timber", "agio": 1, "weight": 83.00782 }, 1010197: { "guid": 1010197, "name": "Wool", "agio": 1, "weight": 28.67188 }, 1010198: { "guid": 1010198, "name": "Red Peppers", "agio": 1, "weight": 244.67188 }, 1010199: { "guid": 1010199, "name": "Pigs", "agio": 1, "weight": 74.01563999999999 }, 1010200: { "guid": 1010200, "name": "Fish", "agio": 1, "weight": 70.43361 }, 1010201: { "guid": 1010201, "name": "Clay", "agio": 1, "weight": 106.46363 }, 1010203: { "guid": 1010203, "name": "Soap", "agio": 1, "weight": 205.39745 }, 1010204: { "guid": 1010204, "name": "Brass", "agio": 1.4, "weight": 201.736355 }, 1010206: { "guid": 1010206, "name": "Sewing Machines", "agio": 1.4, "weight": 812.7386999999999 }, 1010208: { "guid": 1010208, "name": "Light Bulbs", "agio": 1.4, "weight": 1780.6027725 }, 1010209: { "guid": 1010209, "name": "Furs", "agio": 1.2, "weight": 387.67188 }, 1010213: { "guid": 1010213, "name": "Bread", "agio": 1.2, "weight": 269.56109000000004 }, 1010214: { "guid": 1010214, "name": "Beer", "agio": 1.4, "weight": 517.77842 }, 1010215: { "guid": 1010215, "name": "Goulash", "agio": 1.2, "weight": 636.70794 }, 1010216: { "guid": 1010216, "name": "Schnapps", "agio": 1, "weight": 121.20316 }, 1010217: { "guid": 1010217, "name": "Canned Food", "agio": 1.4, "weight": 987.3638699999999 }, 1010219: { "guid": 1010219, "name": "Steel", "agio": 1.2, "weight": 347.01815999999997 }, 1010222: { "guid": 1010222, "name": "Dynamite", "agio": 1, "weight": 1412.0550425000001 }, 1010225: { "guid": 1010225, "name": "Steam Carriages", "agio": 1.6, "weight": 7914.931003 }, 1010226: { "guid": 1010226, "name": "Coal", "agio": 1, "weight": 102.96363 }, 1010227: { "guid": 1010227, "name": "Iron", "agio": 1, "weight": 102.96363 }, 1010228: { "guid": 1010228, "name": "Quartz Sand", "agio": 1, "weight": 80.30454250000001 }, 1010229: { "guid": 1010229, "name": "Zinc", "agio": 1.2, "weight": 72.481815 }, 1010230: { "guid": 1010230, "name": "Copper", "agio": 1.2, "weight": 72.481815 }, 1010231: { "guid": 1010231, "name": "Cement", "agio": 1.2, "weight": 72.481815 }, 1010232: { "guid": 1010232, "name": "Saltpetre", "agio": 1, "weight": 197.30454250000003 }, 1010233: { "guid": 1010233, "name": "Gold Ore", "agio": 1.2, "weight": 149.481815 }, 1010234: { "guid": 1010234, "name": "Tallow", "agio": 1, "weight": 133.85199999999998 }, 1010236: { "guid": 1010236, "name": "Malt", "agio": 1.4, "weight": 219.116485 }, 1010237: { "guid": 1010237, "name": "Work Clothes", "agio": 1, "weight": 90.03128000000001 }, 1010238: { "guid": 1010238, "name": "Sausages", "agio": 1, "weight": 149.56108999999998 }, 1010240: { "guid": 1010240, "name": "Cotton Fabric", "agio": 1.4, "weight": 270.55999599999996 }, 1010241: { "guid": 1010241, "name": "Glass", "agio": 1, "weight": 338.5609425 }, 1010242: { "guid": 1010242, "name": "Wood Veneers", "agio": 1.4, "weight": 724.8750399999999 }, 1010243: { "guid": 1010243, "name": "Filaments", "agio": 1.2, "weight": 775.5027299999999 }, 1010245: { "guid": 1010245, "name": "Penny Farthings", "agio": 1, "weight": 1811.1224479999998 }, 1010246: { "guid": 1010246, "name": "Pocket Watches", "agio": 1.4, "weight": 2434.2299274999996 }, 1010247: { "guid": 1010247, "name": "Fur Coats", "agio": 1.2, "weight": 1162.7446759999998 }, 1010248: { "guid": 1010248, "name": "Gramophones", "agio": 1.4, "weight": 2310.835685 }, 1010249: { "guid": 1010249, "name": "Gold", "agio": 1.6, "weight": 797.894695 }, 1010250: { "guid": 1010250, "name": "Jewellery", "agio": 1.6, "weight": 1524.0192819999997 }, 1010252: { "guid": 1010252, "name": "Tobacco", "agio": 1.2, "weight": 588.879998 }, 1010255: { "guid": 1010255, "name": "Caoutchouc", "agio": 1.4, "weight": 231.87999799999997 }, 1010256: { "guid": 1010256, "name": "Pearls", "agio": 1.4, "weight": 344.239987 }, 1010257: { "guid": 1010257, "name": "Rum", "agio": 1.4, "weight": 267.45593199999996 }, 1010258: { "guid": 1010258, "name": "Chocolate", "agio": 1.2, "weight": 1012.116196 }, 1010259: { "guid": 1010259, "name": "Cigars", "agio": 1.6, "weight": 2083.3539379999997 } }
export_products = {1010224: {"guid": 1010224, "name": "Steam Motors", "weight": 2699.361665}, 1010223: {"guid": 1010223, "name": "Advanced Weapons", "weight": 4097.7803525}, 134623: {"guid": 134623, "name": "Elevators", "weight": 9870},}
products = all_products
w_steam = all_products[1010225]["weight"]

def addDemand(name, amount):
    global demands
    if not name in name_to_guid :
        raise ValueError(name + " not known or cannot be imported")
    
    guid = name_to_guid[name]
    if guid in demands:
        demands[guid] = max(0,  demands[guid] + amount)
    else:
        demands[guid] = max(0, amount)
        
def addExcessGood(name, amount):
    global demands, excessGoods
    if not name in name_to_guid:
        raise ValueError(name + " not known")
    
    guid = name_to_guid[name]
    
    if not guid in products:
        products[guid] = export_products[guid]
        products[guid]['agio'] = 1000
    
    if guid in demands:
        if amount >= demands[guid] :
            amount -= demands[guid]
            del demands
        else :
            demands[guid] -= amount
            return
    
    if guid in excessGoods:
        excessGoods[guid] = max(0, excessGoods[guid] + amount)
    else:
        excessGoods[guid] = max(0, amount) 
    

In [8]:
# set island specific values, e.g. transfer time

global visitInterval, loadingSpeed, totalProfit, totalAmount, duration, storageCapacity
storageCapacity = 10000
visitInterval = 120 + 20 * 60 # seconds
loadingSpeed = 44 * 3 # t/s, Morris has a 200% loading speed buff

# set starting point for optimization - no need to change that
totalProfit = 6174189.609999999 / 1000 * storageCapacity
totalAmount = 1000 * 9 
duration = visitInterval + totalAmount / loadingSpeed
populationFactor = 1

#islands = {"A": {"name": "A", "capacity": 21000, "maxContracts": 4},"B": {"name": "B", "capacity": 5850, "maxContracts": 4} ,"C": {"name": "C", "capacity": 6650, "maxContracts": 4},"D": {"name": "D", "capacity": 6050, "maxContracts": 4}}

In [9]:
# define production / consumption scenario

global demands, population
population = 660000 # all values scale relative to this population count

def generate_demands():
    global demands, population, name_to_guid,sum_demands,excessGoods,summary,demandCarriages
    
    """#toasters, lamps, detergent, no violins
    demandCarriages = 23.76/60 #Steam Carriages
    demands = { # in t / s
        1010248: 1.052631, #Gramophones for Ketema
        120032: 397.56/60, #Coffe
        120030: 75.1/60, # Spectales
        1010208: 69.71/60, #Light Bulbs
        120016: 99.35/60, #Champagne
        1010259: 93.87/60, #Cigars
        1010258: 225.29/60, # Chocolate

        1010192: 103.57/60, # Grain
        1010234: 107.5352/60, # Tallow
        1010219: (59.14+7)/60, # Steel for typewriters and toasters
        1010204: (59.14+7)/60, # Brass for Typewriters and Lamps
        120008: 102.5/60, # Wood for Ethanol
        1010241: 7/60, # glass for Lamps
        1010228: 59.14/60, # Sand for Typewriters
        1010255: 2.18/60, # Caoutchouc for Chewing Gum
        1010232: 7/60, # Salpetre for Detergent 
        1010243: 7/60, # Filaments for Toasters
        1010229: 7/60, # Zinc for Toasters
        120044: 17.74079/60, # Felt
        120034: 68.18959/60, # Corn
    }"""


     
    #vacuum cleaners, lamps, detergent, no violins
    demandCarriages = 28.159972/60 #Steam Carriages
    demands = { # in t / s
        120032: 397.55528/60, #Coffe
        120030: 75.093258/60, # Spectales
        1010208: 105.6/60, #Light Bulbs
        120016: 99.34848/60, #Champagne
        1010259: 93.86657/60, #Cigars
        1010258: 225.28/60, # Chocolate
        
        #1010245: 205.29/60, # Penny Fatherings
        #1010246: 64.41/60, # Pocket Watches
        #1010250: 86.46/60, # Jewellery
        #1010248: 21.61/60, # Phonographen
        
    }
    

    name_to_guid = {}
    for guid in products :
        name_to_guid[products[guid]['name']] = guid

    for guid in export_products:
        name_to_guid[export_products[guid]['name']] = guid

    excessGoods = {}
    summary=[]



generate_demands()

In [9]:
   
# bicycles (clipped)
def add_bicycles_clipped(f):
    # Production (t/min):
    # 11.7 of Bicycles
    #f=11.5
    #f=2.393 # penny farthing production at the space of 140k residents (space in NW free for allocation)
    #f=10.477
    #f=1.04 # penny farthing production at the space of 30k residents (space in NW for vacuum cleaners)
    prodi=11.7
    addDemand("Caoutchouc", f*prodi/60)
    addExcessGood("Penny Farthings", f*prodi/60)
    addExcessGood("Pocket Watches", f*prodi/4/60)
    addExcessGood("Gramophones", f*prodi/4/60)
    addExcessGood("Steam Motors", 2*f*prodi/3/60)
    addExcessGood("Advanced Weapons", 2*f*prodi/3/60)

# bicycles (not clipped)
def add_bicycles(f):

    # Production (t/min):
    # 11.7 of Bicycles
    #f=11.5
    #f=2.393 # penny farthing production at the space of 140k residents (space in NW free for allocation)
    #f=10.477
    #f=10.477 # penny farthing production at the space of 30k residents (space in NW for vacuum cleaners)
    prodi=9
    addDemand("Caoutchouc", f*prodi/60)
    addExcessGood("Penny Farthings", f*prodi/60)
    addExcessGood("Pocket Watches", f*prodi/8/60)
    addExcessGood("Gramophones", f*prodi/8/60)
    addExcessGood("Steam Motors", f*prodi/3/60)
    addExcessGood("Advanced Weapons", f*prodi/3/60)

In [5]:
def defineProblem():
    global prob, profit, variables, profitVariables, amountVariables, sharedAmount, minProfit, excessGoods, demands, status, sumDemands
    sum_demands = sum([products[guid]['weight'] * products[guid]['agio']* demands[guid] for guid in demands])
    prob = LpProblem("Arbitrage", LpMaximize)
      
    if 'variables' in globals():
        old_variables = variables
        
    variables = {}
    profitVariables = []
    amountVariables = {}
    
    # determine products that need to be traded on both islands
    minProfit = 0
    sharedAmount = 0
    for guid in demands:
        if demands[guid] * populationFactor * duration > storageCapacity :
            minProfit += demands[guid] * populationFactor * duration * products[guid]['weight'] * products[guid]['agio'] / 2
            sharedAmount = demands[guid] * populationFactor * duration / 2

    i = 1
    for r in rarities :
        weights = {}
        for guid in products : 
            p = products[guid]
            name = str(guid) + '_' + str(r)
            v = LpVariable(name, cat='Binary') # variable: (product, rarity)
            variables[name] = v
            weights[v] = 1
        name = "pyramid_slot_count_" + str(r)
        
        # Constraint 0: The n-th level of the pyramid must contain at most n products
        prob.constraints[name] = LpConstraint(LpAffineExpression(weights), LpConstraintLE, name, i)
        i = i+1

    for guid in products : 
        weights = {}
        for r in rarities :
            p = products[guid]
            name = str(guid) + '_' + str(r)
            v = variables[name]
            weights[v] = 1

        name = str(guid) + "_once_in_pyramid"
        
        # Constraint 1: Each product must occur at most once in pyramid
        prob.constraints[name] = LpConstraint(LpAffineExpression(weights), LpConstraintLE, name, 1)

    weights_objective = {}
    max_amount = LpVariable(name, 0)
    variables["max_amount"] = max_amount
    loadingAmountWeight = totalProfit  * loadingSpeed /(visitInterval * loadingSpeed + totalAmount) / (visitInterval * loadingSpeed + totalAmount)
    weights_objective[max_amount] = -loadingAmountWeight
    profitWeight = 1 / duration
    
    for island in islands:
        weights_dockland = {}
        slots = {}
        amountVariables[island] = []

        c = islands[island]["capacity"]                
        #print(loadingAmountWeight)

        weights_island_amount = {max_amount: -1}
        for guid in products :         
            for r in rarities :

                p = products[guid]

                for direction in ["export", "import"]:
                    name =  island + "_" + direction + "_" + str(guid) + '_' + str(r)
                    limit = 1
                    if guid in demands and direction == "export" :
                        limit = max(0,1- populationFactor * demands[guid] * duration / c)
                                            
                    v = LpVariable(name, 0, limit) #Each island has two continous variables for each product: export and import in [0,1]
                    variables[name] = v
                    weights_island_amount[v] = c
                    amountVariables[island].append(v)                    

                    # incorporate trade ratios and island storage
                    if direction == "export" :
                        weights_dockland[v] = -r * p["weight"] * c
                    else :
                        weights_dockland[v] = p["weight"] * p["agio"] * c

                    name_slag =  island + "_slag_" + direction + "_" + str(guid) + '_' + str(r)
                    v_slag = LpVariable(name_slag, 0, 1)
                    variables[name_slag] = v_slag 

                    name_bin =  island + "_" + direction + "_binary_" + str(guid) + '_' + str(r)
                    v_bin = LpVariable(name_bin, cat='Binary') # Create binary variable to indicate whether the product is imported, or exported respectively.
                    variables[name_bin] = v_bin
                    slots[v_bin] = 1

                    name = island + "_" + direction + "_ceil_" + str(guid) + "_" + str(r)
                    
                    # Constraint 2: Ensure that the binary import/export variable is 1 if more than 0 tons are imported/exported
                    prob.constraints[name] = LpConstraint(LpAffineExpression({v : 1, v_slag : 1, v_bin:-1}), LpConstraintEQ, name, 0)

                name = island + "_import_export_nand_" + str(guid) + "_" + str(r)
                #Constraint 3: ensure that rarity matches and not both import and export occur
                prob.constraints[name] = LpConstraint(LpAffineExpression({
                    variables[island + "_import_binary_" + str(guid) + '_' + str(r)] : 1, 
                    variables[island + "_export_binary_" + str(guid) + '_' + str(r)] : 1, 
                    variables[str(guid) + '_' + str(r)] : -1, 
                }), LpConstraintLE, name, 0)


        name =  island + "_profit"
        v = LpVariable(name, minProfit)
        variables[name] = v
        profitVariables.append(v)
        weights_dockland[v] = 1
        weights_objective[v] = profitWeight

        name = island + "_import_export_consistency"
        # Constraint 4: Ensure that the summed goods value of import and export at one island is identical.
        prob.constraints[name] = LpConstraint(LpAffineExpression(weights_dockland), LpConstraintEQ, name, 0)

        # Constraint 5: Restrict the maximum number of contracts
        if "maxContracts" in islands[island] :
            name = island + "_maximum_contracts"
            prob.constraints[name] = LpConstraint(LpAffineExpression(slots), LpConstraintLE, name, islands[island]["maxContracts"] + 1)

        name = island + "_summed_amount"
        prob.constraints[name] = LpConstraint(LpAffineExpression(weights_island_amount), LpConstraintLE, name, 0)
        
    for guid in products : 
        for r in rarities :
            weights_global = {}

            p = products[guid]

            for island in islands:
                weights_global[variables[island + "_export_" + str(guid) + '_' + str(r)]] = islands[island]["capacity"]
                weights_global[variables[island + "_import_" + str(guid) + '_' + str(r)]] = -islands[island]["capacity"]

            name = str(guid) + '_' + str(r) + "_good_consistency"
            
            # Constraint 5: Ensure that the sum of imports and exports for one arbitrage goods is the same. 
            # Consumed goods are not considered here.
            if guid in excessGoods :
                # Allow more goods to be exported if we have excess goods
                prob.constraints[name] = LpConstraint(LpAffineExpression(weights_global), LpConstraintLE, name, excessGoods[guid] * duration * populationFactor)
            else :
                rhs = 0
                if guid == 1010225 and  r == 2:
                    rhs = -min(storageCapacity,demandCarriages * duration * populationFactor)
                prob.constraints[name] = LpConstraint(LpAffineExpression(weights_global), LpConstraintEQ, name, rhs)

    
    #for i in ["A_import_120035_1.4", "A_export_120035_1.4"]:
    #    variables[i].setInitialValue(0)
    #    variables[i].fixValue()
    
    #i = "1010248_1.4" #Gramophones
    #variables[i].setInitialValue(1)
    #variables[i].fixValue()
    
    #i = "1010223_1.2" #Advanced Weapons
    #variables[i].setInitialValue(1)
    #variables[i].fixValue()
    
    if 'old_variables' in locals():
        for name in old_variables:
            if not name in variables :
                continue
            v = variables[name]  
            val = value(old_variables[name])
            
            if val == None:
                continue
            
            
            if not v.upBound == None :
                val = min(val, v.upBound)
                
            if not v.lowBound == None:
                val = max(val, v.lowBound)
            
            v.setInitialValue(val)
                
    
    prob.objective = LpAffineExpression(weights_objective)
    
def solve():
    global status
    status = prob.solve(apis.CPLEX_CMD(timeLimit = 120, warmStart=True))
    status = prob.solve(apis.PULP_CBC_CMD(timeLimit = 120, gapAbs=0.1, warmStart=True))
    #status = prob.solve(apis.COIN_CMD(timeLimit = 120, gapAbs=0.001, warmStart=True, threads=8))
    #print(LpStatus[status])
    
def optimize():
    defineProblem()
    solve()

def fixPyramid(pyramid):
    for r in range(len(pyramid)):
        for name in pyramid[r] :
            n = str(name_to_guid[name]) + "_" + str(rarities[r])
            variables[n].setInitialValue(1)
            variables[n].fixValue()
    
def update_variables(logStatus = True):
    global totalProfit,totalAmount,duration,profitPerMinute,profit,populationFactor,sharedAmount,status,error
    oldDuration = duration
    totalProfit = sum([value(v) for v in profitVariables]) #+ sum([products[guid]["weight"] * min(storageCapacity,demands[guid] * duration ) for guid in demands])
    totalAmount = max([sum([abs(value(v)) * islands[island]["capacity"] for v in amountVariables[island]]) for island in islands]) +  sum([demands[guid] * duration * populationFactor  for guid in demands]) - sharedAmount
    duration = visitInterval + totalAmount / loadingSpeed  
    error = abs(1-duration/oldDuration)
    sum_demands = sum([products[guid]['weight'] * products[guid]['agio']* demands[guid] for guid in demands])
    populationFactor = totalProfit / duration / sum_demands
    profitPerMinute = totalProfit / duration * 60
    profit = totalProfit / max([islands[island]["capacity"] for island in islands])
    if logStatus :
        print("{}, Profit {:.2f} t/min, {:.2f} min, Error {:.2%}".format(LpStatus[status], profitPerMinute/2/w_steam, duration/60, error))

def print_result():
    global summary
    min_weight_good = 0
    min_weight_good_weight = 999999999999
    min_weight_good_rarity = 0
    
    slots = 0

    #print results
    print()
    print("Pyramid:")
    i = 1
    for r in rarities :
        output = ""
        for v in prob.constraints["pyramid_slot_count_" + str(r)].toDict()['coefficients']:
            guid = int(v['name'].split('_')[0])
            if value(variables[v['name']]) > 0 and sum([value(variables[island + "_export_" + str(guid) + "_" + str(r)]) for island in islands]) > 0:
                slots += 1
                p = products[guid]
                if p['weight'] < min_weight_good_weight:
                    min_weight_good = p
                    min_weight_good_weight = p['weight']
                    min_weight_good_rarity = r

                if output:
                    output += ", "
                output += p['name']
        output = str(i) + ": " + output
        print(output)
        i = i+1

    print()

    contracts = {}
    for island in islands:
        contracts[island] = {"import": "", "export":""}


    for v in variables:
        if value(variables[v]) == 0:
            continue

        name = v.split('_')[0]
        direction = v.split('_')[1]

        if direction == "export" or direction == "import":
            guid = v.split('_')[2]
            if guid == 'binary':
                continue


                
            if contracts[name][direction]:
                contracts[name][direction] += ", "

            contracts[name][direction] += products[int(guid)]['name'] +  " (" + str(value(variables[v]) * islands[name]['capacity']) + "t)"
        if direction == "profit":
            contracts[name]["profit"] =  value(variables[v])

    for island in islands:
        if value(variables[island + "_import_1010225_2"]) > 0 :
            for guid in demands :
                if demands[guid] * populationFactor * duration > storageCapacity :
                    contracts[island]["import"] += ", " + products[int(guid)]['name'] +  " (" + str(demands[guid] * populationFactor * duration / 2) + "t)"
        else :
            for guid in demands :
              
                amount = demands[guid] * duration * populationFactor

                if amount > storageCapacity:
                    amount /= 2

                contracts[island]["import"] += ", " + products[int(guid)]['name'] +  " (" + str(amount) + "t)"
        
        print("Island " + island + ":")
        print("Export: " + contracts[island]["export"])
        print("Import: " + contracts[island]["import"])
        if "profit" in contracts[island]:
            print("Profit: " + str(contracts[island]["profit"]))
        print()

    #print("Total profit: " + str(profitPerMinute) + " or " + str(profitPerMinute/2/w_steam) + " t of legendary Steam Carriages per minute")
    #print("Trading interval: " + str(duration / 60) + " min")
    #print()
    #print("Supplied population: " + str(population * populationFactor))

    totalProfit_ = totalProfit - (storageCapacity * min_weight_good['weight'] * min_weight_good_rarity / 1.6 * 2- storageCapacity * min_weight_good['weight'] * min_weight_good['agio'])
    #print(totalProfit)
    #print(totalProfit_)
    totalAmount_ = totalAmount - storageCapacity
    duration_ = visitInterval + totalAmount_ / loadingSpeed   
    profitPerMinute_ = totalProfit_ / duration_ * 60
    #print("With one slot less: "+str(profitPerMinute_/2/w_steam) + " t/min of legendary Steam Carriages, trading every " + str(duration_ / 60) + " min")
    
    #print()
    #smallIslandStorage = storageCapacity / duration *(visitInterval + (sharedAmount + min([sum([abs(value(v)) * islands[island]["capacity"] for v in amountVariables[island]]) for island in islands])) / loadingSpeed   )
    x = (sharedAmount + min([sum([abs(value(v)) * islands[island]["capacity"] for v in amountVariables[island]]) for island in islands])) /duration / loadingSpeed
    smallIslandStorage =storageCapacity / duration  * visitInterval *(1-x/(x-1))
    minGoodWeight = profitPerMinute / 1.2 / loadingSpeed / 60
    summary.append(str(storageCapacity) +  "\tH\t" + str(profit) + "\t" + str(profitPerMinute/2/w_steam) + "\t" + str(duration/60) + "\t" + str(slots-1) + "\t" + str(population * populationFactor) + "\t" + str(smallIslandStorage) + "\t" + str(minGoodWeight))
    print(summary[-1])

In [7]:
def supplyable_population(factories):
    generate_demands()
    add_bicycles_and_elevators(factories)
    for i in range(7) :
        defineProblem()
        #fixPyramid(pyramid)
        solve()
        update_variables(False)
        if error < 0.0001:
            break
    return population * populationFactor



def estimate_factories(lower_init = 0, upper_init = 20):
    epsilon =  1000
    
    #base = {'factories': 0, 'pop': supplyable_population(0)}
    #if base['pop'] > population :
        #return 0
    
    lower = {'factories': lower_init, 'pop': supplyable_population(lower_init)}

    
    upper = {'factories': upper_init, 'pop': supplyable_population(upper_init)} 
    while True:
        center = lower['factories']+(upper['factories']-lower['factories'])*(targetPopulation  - lower['pop'])/(upper['pop'] - lower['pop'])

        print(lower['factories'], lower['pop'],upper['factories'], upper['pop'], center)
        
        
        if abs(lower['pop']-targetPopulation) < epsilon or abs(upper['pop']-targetPopulation) < epsilon :
            return center
        
        pop = supplyable_population(center)
        if pop > upper['pop']:
            upper = {'factories': center, 'pop': pop}
        elif pop < lower['pop']:
            lower = {'factories': center, 'pop': pop}
        elif pop > targetPopulation:
            upper = {'factories': center, 'pop': pop}
        else :
            lower = {'factories': center, 'pop': pop}

## Calculate the best pyramid for an arbitrage system consisting of two islands

In [12]:
storageCapacity = 6000
islands = {"A": {"name": "A", "capacity": storageCapacity, "maxContracts": 8},"B": {"name": "B", "capacity": storageCapacity, "maxContracts": 8} }
for i in range(7) :
    defineProblem()
    #fixPyramid(pyramid)
    solve()
    update_variables(True)

print_result()


Optimal, Profit 57.92 t/min, 30.02 min, Error 0.31%
Optimal, Profit 60.84 t/min, 29.76 min, Error 0.86%
Optimal, Profit 60.03 t/min, 29.83 min, Error 0.24%
Optimal, Profit 60.25 t/min, 29.82 min, Error 0.06%
Optimal, Profit 60.19 t/min, 29.82 min, Error 0.02%
Optimal, Profit 60.21 t/min, 29.82 min, Error 0.00%
Optimal, Profit 60.21 t/min, 29.82 min, Error 0.00%

Pyramid:
1: Steam Carriages
2: Penny Farthings, Pocket Watches
3: Tailored Suits, Dynamite, Fur Coats
4: Caribou Meat, Tortillas

Island A:
Export: Caribou Meat (6000.0t), Tailored Suits (6000.0t), Tortillas (6000.0t), Dynamite (6000.0t), Penny Farthings (6000.0t), Pocket Watches (6000.0t), Fur Coats (6000.0t)
Import: Steam Carriages (6000.0t), Coffee (3469.539018296721t)
Profit: 5457725.9

Island B:
Export: Steam Carriages (5508.45504t)
Import: Caribou Meat (6000.0t), Tailored Suits (6000.0t), Tortillas (6000.0t), Dynamite (6000.0t), Penny Farthings (6000.0t), Pocket Watches (6000.0t), Fur Coats (6000.0t), Coffee (3469.5390182

## Calculate the best arbitrage for storage capacities from 4kt to 16kt

In [13]:

# If the error does not reach 0.00%, the optimisation oscillated between several solutions and the output is not valid

global storageCapacity, summary
summary = []
for s in range(4000,14250,250):
    storageCapacity = s
    islands = {"A": {"name": "A", "capacity": storageCapacity, "maxContracts": 8},"B": {"name": "B", "capacity": storageCapacity, "maxContracts": 8} }
    for i in range(7) :
        defineProblem()
        #fixPyramid(pyramid)
        solve()
        update_variables(True)

    print_result()

for entry in summary:
    print(entry)

Optimal, Profit 37.03 t/min, 27.90 min, Error 6.45%
Optimal, Profit 45.92 t/min, 27.17 min, Error 2.61%
Optimal, Profit 43.39 t/min, 27.26 min, Error 0.35%
Optimal, Profit 44.14 t/min, 27.20 min, Error 0.23%
Optimal, Profit 43.94 t/min, 27.22 min, Error 0.06%
Optimal, Profit 43.99 t/min, 27.21 min, Error 0.02%
Optimal, Profit 43.98 t/min, 27.21 min, Error 0.00%

Pyramid:
1: Steam Carriages
2: Penny Farthings, Pocket Watches
3: Tailored Suits, Dynamite, Fur Coats
4: Caribou Meat, Tortillas

Island A:
Export: Steam Carriages (3672.2411199999997t)
Import: Caribou Meat (4000.0t), Tailored Suits (4000.0t), Tortillas (4000.0t), Dynamite (4000.0t), Penny Farthings (4000.0t), Pocket Watches (4000.0t), Fur Coats (4000.0t), Coffee (2312.90571595449t), Glasses (873.7583646623697t), Light Bulbs (1228.72393295742t), Champagne (1155.9834761263405t), Cigars (1092.1979267388538t), Chocolate (2621.277723642496t)
Profit: 15306914.0

Island B:
Export: Caribou Meat (4000.0t), Tailored Suits (4000.0t), Tor